In [1]:
import torch
import onnxruntime
import numpy as np
import cv2
from albumentations.pytorch import ToTensorV2
import albumentations as A
import onnxruntime as onnxrt

C:\Users\likeo\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def pre_process(numpy_data):
    transform = A.Compose(
                [
                    A.Resize(height=224, width=224),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255., 
                        p=1.0
                    ),
                    ToTensorV2(),
                ])
    transform = transform(image=numpy_data)
    image = transform["image"]
    return image
    
def predict(numpy_data):
    input_data = pre_process(numpy_data)
    input_batch = input_data.unsqueeze(0)
    with open('imagenet-idx.txt') as f:
        classes = [line.strip() for line in f.readlines()]

    onnx_session= onnxrt.InferenceSession("models\\tensorrt_fp16_model\\1\\model.onnx")
    onnx_output = onnx_session.run(None, {'input' : input_batch.numpy().astype(np.float16)})
    onnx_output = torch.FloatTensor(np.array(onnx_output)).reshape(1,-1)
    prob = torch.nn.functional.softmax(onnx_output, dim=1)[0] * 100
    _, indices = torch.sort(prob, descending=True)
    return [(classes[idx], prob[idx].item()) for idx in indices[:10]]

In [9]:
image = cv2.imread('images\\toy.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
transformed_image = pre_process(image)

In [10]:
pred = predict(image)

In [11]:
pred

[('toy poodle', 79.48882293701172),
 ('miniature poodle', 20.097919464111328),
 ('teddy, teddy bear', 0.32996830344200134),
 ('standard poodle', 0.0419507659971714),
 ('Airedale, Airedale terrier', 0.013619418255984783),
 ('Norwich terrier', 0.007699721958488226),
 ('Irish terrier', 0.0031599593348801136),
 ('Yorkshire terrier', 0.002854785416275263),
 ('Lakeland terrier', 0.0024039701092988253),
 ('wig', 0.0016013894928619266)]